In [84]:
import requests
import pandas as pd
import json

import datetime
import os
from dotenv import load_dotenv, find_dotenv


dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

True

In [51]:
FACEIT_PLAYERID = os.environ.get("FACEIT_PLAYERID")
KEY = os.environ.get("KEY")
HEADER = {"Authorization": f"Bearer {KEY}"}
BASIC_URL = "https://open.faceit.com/data/v4/"

In [107]:
def shorter_url(url = "") -> dict:
    request = BASIC_URL + url
    return requests.get(request, headers = HEADER).json()

def retreive_player_by_id(id: str) -> dict:
    response = shorter_url(url=f"players/{id}")
    return response

def retreive_player_game_stats(id: str) -> dict:
    response = shorter_url(url=f"players/{id}/games/csgo/stats")
    return response

def get_number_of_games_played(id: str) -> int:
    response = int(shorter_url(url = f"players/{id}/stats/csgo")["lifetime"]["Matches"])
    return response

def retreive_player_stats_overall(id: str) -> dict:
    response = shorter_url(url = f"players/{id}")
    return response

def get_time_registered(id: str) -> str:
    response = retreive_player_stats_overall(id)
    return response["activated_at"]

def retreive_player_history(id: str, offset = 0, limit = 100) -> dict:
    games_played = get_number_of_games_played(id)
    registered = get_time_registered(id)
    registered = int(pd.Timestamp(registered).timestamp())
    offset = 0
    times_to_run = games_played // limit
    response_array = []

    while times_to_run > 0:
        response = shorter_url(url = f"players/{id}/history?game=csgo&from={registered}&offset={offset}&limit={limit}")["items"]
        response_array.extend(response)
        times_to_run-=1
        offset+=limit
        
    else:
        response = shorter_url(url = f"players/{id}/history?game=csgo&from={registered}&offset={offset}&limit={limit}")["items"]
        response_array.extend(response[1::])
    return response_array